In [19]:
# import api keys
from api_keys import *

## Flights API

In [2]:
from datetime import datetime, timedelta

airport_icoa = "EDDB"
to_local_time = datetime.now().strftime('%Y-%m-%dT%H:00')
from_local_time = (datetime.now() + timedelta(hours=9)).strftime('%Y-%m-%dT%H:00')
url = f"https://aerodatabox.p.rapidapi.com/flights/airports/icao/{airport_icoa}/{to_local_time}/{from_local_time}"

import requests

querystring = {"withLeg":"true","withCancelled":"true","withCodeshared":"true","withCargo":"true","withPrivate":"false","withLocation":"false"}

headers = {
    'x-rapidapi-host': "aerodatabox.p.rapidapi.com",
    'x-rapidapi-key': flight_api_key
    }

response = requests.request("GET", url, headers=headers, params=querystring)

from IPython.display import JSON
response.json()

{'departures': [{'departure': {'scheduledTimeLocal': '2022-08-09 14:15+02:00',
    'scheduledTimeUtc': '2022-08-09 12:15Z',
    'terminal': '1',
    'quality': ['Basic']},
   'arrival': {'airport': {'icao': 'BIKF', 'iata': 'KEF', 'name': 'Reykjavik'},
    'scheduledTimeLocal': '2022-08-09 15:50+00:00',
    'scheduledTimeUtc': '2022-08-09 15:50Z',
    'quality': ['Basic']},
   'number': 'FI 529',
   'status': 'Unknown',
   'codeshareStatus': 'Unknown',
   'isCargo': False,
   'aircraft': {'model': 'Boeing 767-300'},
   'airline': {'name': 'Icelandair'}},
  {'departure': {'scheduledTimeLocal': '2022-08-09 13:10+02:00',
    'scheduledTimeUtc': '2022-08-09 11:10Z',
    'terminal': '1',
    'quality': ['Basic']},
   'arrival': {'airport': {'icao': 'DTMB', 'iata': 'MIR', 'name': 'Monastir'},
    'scheduledTimeLocal': '2022-08-09 14:50+01:00',
    'actualTimeLocal': '2022-08-09 14:50+01:00',
    'scheduledTimeUtc': '2022-08-09 13:50Z',
    'actualTimeUtc': '2022-08-09 13:50Z',
    'baggageBel

In [3]:
### Option 1
arrivals_berlin = response.json()['arrivals']

def get_flight_info(flight_json):
    # terminal
    try: terminal = flight_json['arrival']['terminal']
    except: terminal = None
    # aircraft
    try: aircraft = flight_json['aircraft']['model']
    except: aircraft = None

    return {
        'dep_airport':flight_json['departure']['airport']['name'],
        'sched_arr_loc_time':flight_json['arrival']['scheduledTimeLocal'],
        'terminal':terminal,
        'status':flight_json['status'],
        'aircraft':aircraft,
        'icao_code':airport_icoa
    }


import pandas as pd
# [get_flight_info(flight) for flight in arrivals_berlin]
arrivals_berlin = pd.DataFrame([get_flight_info(flight) for flight in arrivals_berlin])
arrivals_berlin

,dep_airport,sched_arr_loc_time,terminal,status,aircraft,icao_code
0,Helsinki,2022-08-09 12:25+02:00,1,Unknown,Airbus A319,EDDB
1,Monastir,2022-08-09 12:10+02:00,1,Unknown,Airbus A320,EDDB
2,Barcelona,2022-08-09 12:20+02:00,0,Unknown,Boeing 737-800,EDDB
3,Valencia,2022-08-09 12:25+02:00,0,Unknown,Boeing 737-800,EDDB
4,Palma De Mallorca,2022-08-09 12:05+02:00,0,Unknown,Boeing 737-800,EDDB
...,...,...,...,...,...,...
102,Frankfurt-am-Main,2022-08-09 20:55+02:00,1,Unknown,Airbus A320,EDDB
103,Nice,2022-08-09 20:20+02:00,1,Unknown,Airbus A320,EDDB
104,Bastia/Poretta,2022-08-09 20:25+02:00,1,Unknown,Airbus A319,EDDB
105,London,2022-08-09 20:05+02:00,1,Unknown,Airbus A320,EDDB


## Wheather API

In [20]:
city = "Berlin"
country = "DE"
response = requests.get(f'http://api.openweathermap.org/data/2.5/forecast/?q={city},{country}&appid={OWM_key}&units=metric&lang=en')
from IPython.display import JSON

# JSON(response.json())
response.status_code

200

In [5]:
forecast_api = response.json()['list']
# look for the fields that could be relevant: 
# better field descriptions https://www.weatherbit.io/api/weather-forecast-5-day

weather_info = []

# datetime, temperature, wind, prob_perc, rain_qty, snow = [], [], [], [], [], []
for forecast_3h in forecast_api: 
    weather_hour = {}
    # datetime utc
    weather_hour['datetime'] = forecast_3h['dt_txt']
    # temperature 
    weather_hour['temperature'] = forecast_3h['main']['temp']
    # wind
    weather_hour['wind'] = forecast_3h['wind']['speed']
    # probability precipitation 
    try: weather_hour['prob_perc'] = float(forecast_3h['pop'])
    except: weather_hour['prob_perc'] = 0
    # rain
    try: weather_hour['rain_qty'] = float(forecast_3h['rain']['3h'])
    except: weather_hour['rain_qty'] = 0
    # wind 
    try: weather_hour['snow'] = float(forecast_3h['snow']['3h'])
    except: weather_hour['snow'] = 0
    weather_hour['municipality_iso_country'] = city + ',' + country
    weather_info.append(weather_hour)
    
weather_data = pd.DataFrame(weather_info)
weather_data.head()

,datetime,temperature,wind,prob_perc,rain_qty,snow,municipality_iso_country
0,2022-08-09 12:00:00,25.43,2.55,0.0,0.0,0,"Berlin,DE"
1,2022-08-09 15:00:00,26.46,2.74,0.0,0.0,0,"Berlin,DE"
2,2022-08-09 18:00:00,23.86,3.90,0.0,0.0,0,"Berlin,DE"
3,2022-08-09 21:00:00,19.93,3.98,0.0,0.0,0,"Berlin,DE"
4,2022-08-10 00:00:00,17.65,2.62,0.0,0.0,0,"Berlin,DE"


## Population data

In [6]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import unicodedata

cities = ['Berlin','Paris','Amsterdam','Barcelona','Rome','Lisbon','Prague','Vienna','Madrid']

def City_info(soup):
    
    ret_dict = {}
    ret_dict['city'] = soup.h1.get_text()
    
    if soup.select_one('.mergedrow:-soup-contains("Mayor")>.infobox-label') != None:
        i = soup.select_one('.mergedrow:-soup-contains("Mayor")>.infobox-label')
        mayor_name_html = i.find_next_sibling()
        mayor_name = unicodedata.normalize('NFKD',mayor_name_html.get_text())
        ret_dict['mayor']  = mayor_name
    
    if soup.select_one('.mergedrow:-soup-contains("City")>.infobox-label') != None:
        j =  soup.select_one('.mergedrow:-soup-contains("City")>.infobox-label')
        area = j.find_next_sibling('td').get_text()
        ret_dict['city_size'] = unicodedata.normalize('NFKD',area)

    if soup.select_one('.mergedtoprow:-soup-contains("Elevation")>.infobox-data') != None:
        k = soup.select_one('.mergedtoprow:-soup-contains("Elevation")>.infobox-data')
        elevation_html = k.get_text()
        ret_dict['elevation'] = unicodedata.normalize('NFKD',elevation_html)
    
    if soup.select_one('.mergedtoprow:-soup-contains("Population")') != None:
        l = soup.select_one('.mergedtoprow:-soup-contains("Population")')
        c_pop = l.findNext('td').get_text()
        ret_dict['city_population'] = c_pop
    
    if soup.select_one('.infobox-label>[title^=Urban]') != None:
        m = soup.select_one('.infobox-label>[title^=Urban]')
        u_pop = m.findNext('td')
        ret_dict['urban_population'] = u_pop.get_text()

    if soup.select_one('.infobox-label>[title^=Metro]') != None:
        n = soup.select_one('.infobox-label>[title^=Metro]')
        m_pop = n.findNext('td')
        ret_dict['metro_population'] = m_pop.get_text()
    
    if soup.select_one('.latitude') != None:
        o = soup.select_one('.latitude')
        ret_dict['lat'] = o.get_text()

    if soup.select_one('.longitude') != None:    
        p = soup.select_one('.longitude')
        ret_dict['long'] = p.get_text()
    
    return ret_dict

list_of_city_info = []
for city in cities:
    url = 'https://en.wikipedia.org/wiki/{}'.format(city)
    web = requests.get(url,'html.parser')
    soup = bs(web.content)
    list_of_city_info.append(City_info(soup))
df_cities = pd.DataFrame(list_of_city_info)
# df_cities = df_cities.set_index('city')
df_cities

,city,mayor,city_size,elevation,city_population,urban_population,metro_population,lat,long
0,Berlin,Franziska Giffey (SPD),891.7 km2 (344.3 sq mi),34 m (112 ft),"3,769,495","4,473,101","6,144,600",52°31′12″N,13°24′18″E
1,Paris,Anne Hidalgo (PS),NaN,28–131 m (92–430 ft) (avg. 78 m or 256 ft),"2,165,423","10,858,852","13,024,518",48°51′24″N,2°21′08″E
2,Amsterdam,Femke Halsema (GL),NaN,−2 m (−7 ft),"907,976","1,558,755",NaN,52°22′N,4°54′E
3,Barcelona,Ada Colau Ballano[1] (Barcelona en Comú),101.4 km2 (39.2 sq mi),12 m (39 ft),"1,620,343","4,840,000[3]","5,474,482[4]",41°23′N,2°11′E
4,Rome,Strong Mayor–Council,"4,342,212[2]",21 m (69 ft),1st in Italy (3rd in the EU),NaN,Rome Capital,41°53′36″N,12°28′58″E
5,Lisbon,Carlos Moedas,NaN,2 m (7 ft),"544,851[1]","2,719,000[4]","2,871,133[2][3]",38°43′31″N,9°09′00″W
6,Prague,Zdeněk Hřib (Pirates),NaN,NaN,"1,275,406",NaN,"2,709,418[4]",50°05′15″N,14°25′17″E
7,Vienna,Michael Ludwig (SPÖ),NaN,"151 (Lobau) – 542 (Hermannskogel) m (495–1,778...",1st in Austria (6th in EU),"1,951,354","2,890,577",48°12′N,16°22′E
8,Madrid,José Luis Martínez-Almeida (PP),NaN,"650 m (2,130 ft)","3,223,334","6,211,000[2]","6,791,667[1]",40°25′00″N,03°42′09″W


In [7]:
df_cities['municipality_iso_country'] = [
    'Berlin,DE',
    'Paris,FR',
    'Amsterdam,NL',
    'Barcelona,ES',
    'Rome,IT',
    'Lisbon,PT',
    'Prague,CZE',
    'Vienna,AT',
    'Madrid,ES'
]

## Airports data

In [8]:
import pandas as pd

airports_cities = (
pd.read_csv('airports.csv')
    .query('type == "large_airport"')
    .filter(['name','latitude_deg','longitude_deg','iso_country','iso_region','municipality','gps_code','iata_code'])
    .rename(columns={'gps_code':'icao_code'})
    .assign(municipality_iso_country = lambda x: x['municipality'] + ',' + x['iso_country'])
)
airports_cities.head()

,name,latitude_deg,longitude_deg,iso_country,iso_region,municipality,icao_code,iata_code,municipality_iso_country
10890,Honiara International Airport,-9.428000,160.054993,SB,SB-CT,Honiara,AGGH,HIR,"Honiara,SB"
12461,Port Moresby Jacksons International Airport,-9.443380,147.220001,PG,PG-NCD,Port Moresby,AYPY,POM,"Port Moresby,PG"
12981,Keflavik International Airport,63.985001,-22.605600,IS,IS-2,Reykjavík,BIKF,KEF,"Reykjavík,IS"
13028,Priština Adem Jashari International Airport,42.572800,21.035801,XK,XK-01,Prishtina,BKPR,PRN,"Prishtina,XK"
17254,Guodu Air Base,36.001741,117.632010,CN,CN-37,"Xintai, Tai'an",NaN,NaN,"Xintai, Tai'an,CN"


## Check the tables

In [9]:
arrivals_berlin.head()

,dep_airport,sched_arr_loc_time,terminal,status,aircraft,icao_code
0,Helsinki,2022-08-09 12:25+02:00,1,Unknown,Airbus A319,EDDB
1,Monastir,2022-08-09 12:10+02:00,1,Unknown,Airbus A320,EDDB
2,Barcelona,2022-08-09 12:20+02:00,0,Unknown,Boeing 737-800,EDDB
3,Valencia,2022-08-09 12:25+02:00,0,Unknown,Boeing 737-800,EDDB
4,Palma De Mallorca,2022-08-09 12:05+02:00,0,Unknown,Boeing 737-800,EDDB


In [10]:
weather_data.head()

,datetime,temperature,wind,prob_perc,rain_qty,snow,municipality_iso_country
0,2022-08-09 12:00:00,25.43,2.55,0.0,0.0,0,"Berlin,DE"
1,2022-08-09 15:00:00,26.46,2.74,0.0,0.0,0,"Berlin,DE"
2,2022-08-09 18:00:00,23.86,3.90,0.0,0.0,0,"Berlin,DE"
3,2022-08-09 21:00:00,19.93,3.98,0.0,0.0,0,"Berlin,DE"
4,2022-08-10 00:00:00,17.65,2.62,0.0,0.0,0,"Berlin,DE"


In [11]:
df_cities.head()

,city,mayor,city_size,elevation,city_population,urban_population,metro_population,lat,long,municipality_iso_country
0,Berlin,Franziska Giffey (SPD),891.7 km2 (344.3 sq mi),34 m (112 ft),"3,769,495","4,473,101","6,144,600",52°31′12″N,13°24′18″E,"Berlin,DE"
1,Paris,Anne Hidalgo (PS),NaN,28–131 m (92–430 ft) (avg. 78 m or 256 ft),"2,165,423","10,858,852","13,024,518",48°51′24″N,2°21′08″E,"Paris,FR"
2,Amsterdam,Femke Halsema (GL),NaN,−2 m (−7 ft),"907,976","1,558,755",NaN,52°22′N,4°54′E,"Amsterdam,NL"
3,Barcelona,Ada Colau Ballano[1] (Barcelona en Comú),101.4 km2 (39.2 sq mi),12 m (39 ft),"1,620,343","4,840,000[3]","5,474,482[4]",41°23′N,2°11′E,"Barcelona,ES"
4,Rome,Strong Mayor–Council,"4,342,212[2]",21 m (69 ft),1st in Italy (3rd in the EU),NaN,Rome Capital,41°53′36″N,12°28′58″E,"Rome,IT"


In [12]:
airports_cities.head()

,name,latitude_deg,longitude_deg,iso_country,iso_region,municipality,icao_code,iata_code,municipality_iso_country
10890,Honiara International Airport,-9.428000,160.054993,SB,SB-CT,Honiara,AGGH,HIR,"Honiara,SB"
12461,Port Moresby Jacksons International Airport,-9.443380,147.220001,PG,PG-NCD,Port Moresby,AYPY,POM,"Port Moresby,PG"
12981,Keflavik International Airport,63.985001,-22.605600,IS,IS-2,Reykjavík,BIKF,KEF,"Reykjavík,IS"
13028,Priština Adem Jashari International Airport,42.572800,21.035801,XK,XK-01,Prishtina,BKPR,PRN,"Prishtina,XK"
17254,Guodu Air Base,36.001741,117.632010,CN,CN-37,"Xintai, Tai'an",NaN,NaN,"Xintai, Tai'an,CN"


In [ ]:
# airports_cities.merge(arrivals_berlin, on='icao_code', how='inner').merge(weather_data, on='municipality_iso_country', how='inner').head()

## Update data into database

First run this code in mysql:

In [21]:
'''
DROP DATABASE gans;
CREATE DATABASE IF NOT EXISTS gans; 
USE gans;

DROP TABLE IF EXISTS cities;
CREATE TABLE IF NOT EXISTS cities (
    city VARCHAR(200),
    mayor TEXT,
    city_size TEXT, 
    elevation TEXT, 
    city_population TEXT, 
    urban_population TEXT, 
    metro_population TEXT, 
    latitude TEXT, 
    longitude TEXT, 
	municipality_iso_country varchar(200),
    PRIMARY KEY(municipality_iso_country)
); 

DROP TABLE IF EXISTS airports;
CREATE TABLE IF NOT EXISTS airports(
	name text, 
    latitude_deg float, 
    longitude_deg float, 
    iso_country varchar(10), 
    iso_region varchar(10),
    municipality text, 
    icao_code varchar(4), 
    iata_code varchar(6), 
    municipality_iso_country varchar(200),
    primary key(icao_code)
    -- foreign key (municipality_iso_country) references cities(municipality_iso_country)
);


DROP TABLE IF EXISTS weather; 
CREATE TABLE IF NOT EXISTS weather (
	weather_id int auto_increment, 
    datetime datetime, 
    temperature float, 
    wind float, 
    prob_perc float, 
    rain_qty float, 
    snow integer, 
    municipality_iso_country varchar(200),
    primary key(weather_id),
    foreign key (municipality_iso_country) references cities(municipality_iso_country)
);

DROP TABLE IF EXISTS arrivals; 
CREATE TABLE IF NOT EXISTS arrivals(
	arrivals_id int auto_increment, 
    dep_airport text, 
    sched_arr_loc_time datetime, 
    terminal text, 
    status text, 
	aircraft text, 
    icao_code varchar(4),
    primary key (arrivals_id)
    -- foreign key (icao_code) references airports(icao_code)
);


USE gans;
SELECT * FROM arrivals;
SELECT * FROM weather;
SELECT * FROM cities;
SELECT * FROM airports;
'''


'\nDROP DATABASE gans;\nCREATE DATABASE IF NOT EXISTS gans; \nUSE gans;\n\nDROP TABLE IF EXISTS cities;\nCREATE TABLE IF NOT EXISTS cities (\n    city VARCHAR(200),\n    mayor TEXT,\n    city_size TEXT, \n    elevation TEXT, \n    city_population TEXT, \n    urban_population TEXT, \n    metro_population TEXT, \n    latitude TEXT, \n    longitude TEXT, \n\tmunicipality_iso_country varchar(200),\n    PRIMARY KEY(municipality_iso_country)\n); \n\nDROP TABLE IF EXISTS airports;\nCREATE TABLE IF NOT EXISTS airports(\n\tname text, \n    latitude_deg float, \n    longitude_deg float, \n    iso_country varchar(10), \n    iso_region varchar(10),\n    municipality text, \n    icao_code varchar(4), \n    iata_code varchar(6), \n    municipality_iso_country varchar(200),\n    primary key(icao_code)\n    -- foreign key (municipality_iso_country) references cities(municipality_iso_country)\n);\n\n\nDROP TABLE IF EXISTS weather; \nCREATE TABLE IF NOT EXISTS weather (\n\tweather_id int auto_increment,

### `sqlalchemy`

#### Establish the connection

In [13]:
import pandas as pd
import sqlalchemy

## If you are running locally:

In [ ]:
# my local database
schema="gans"
host="127.0.0.1"
user="root"
password="password"
port=3306
con = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

## If you are running on the AWS instance:

In [14]:
# my AWS instance
schema="gans"
host=""
user="admin"
password="password"
port=3306
con = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

#### Update the tables

In [15]:
(
df_cities
    # .dropna()
    .rename(
        columns={
            'lat':'latitude',
            'long':'longitude'
            }
        )
    .to_sql('cities', con=con, if_exists='append', index=False)
    )

9

In [16]:
(
airports_cities
    .dropna()
    .to_sql('airports', if_exists='append', con=con, index=False)
)

611

In [17]:
(
    weather_data
    .assign(datetime = lambda x: pd.to_datetime(x['datetime']))
    .to_sql('weather', if_exists='append', con=con, index=False)
)

40

In [18]:
import numpy as np
(
arrivals_berlin
    .replace({np.nan},'unknown')
    .assign(sched_arr_loc_time = lambda x: pd.to_datetime(x['sched_arr_loc_time']))
    .to_sql('arrivals', if_exists='append', con=con, index=False))

107